# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, r2_score

In [3]:
from explainerdashboard.explainers import *
from explainerdashboard.dashboards import *
from explainerdashboard.datasets import *

# RandomForestRegressor

In [18]:
X_train, y_train, X_test, y_test = titanic_fare()
train_names, test_names = titanic_names()

model = RandomForestRegressor(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

explainer = RandomForestRegressionBunch(model, X_test, y_test, r2_score, 
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names, units="$")

In [21]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True,
                        shadow_trees=True)
db.run(8052)

Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Feb/2020 22:21:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:09] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:09] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:09] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:09] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1566138672 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:09] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.min.js?v=0.7.0&m=1566138674 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:09] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.1.1&m=1566138668 HTTP/1.1

# RandomForestClassifier:

In [22]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

model = RandomForestClassifier(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

explainer = RandomForestClassifierBunch(model, X_test, y_test, roc_auc_score, 
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names, #names of passengers 
                               labels=['Not survived', 'Survived'])

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True,
                        shadow_trees=True)
db.run(8052)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating ranks...
Calculating prediction probabilities...
Calculating importances...
Calculating shap interaction values...

                so no dtreeviz visualisation of decision trees will be shown on the shadow trees tab.

                See https://github.com/parrt/dtreeviz for info on how to properly install graphviz 
                for dtreeviz. 
                
Generating shadow trees...
Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Feb/2020 22:21:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:49] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:49] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:49] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:49] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1566138672 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:49] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.min.js?v=0.7.0&m=1566138674 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:49] "GET /_dash-component-suites/dash_table/bundle.js?v=4.1.0&m=1566138672 HTTP/1.1" 200 -
127.0.0.1 

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1287, in dispatch
    response.set_data(self.callback_map[output]['call

127.0.0.1 - - [01/Feb/2020 22:21:57] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Feb/2020 22:21:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 22:21:58] "POST /_dash-update-component HTTP/1.1" 200 -


# XGBoostClassifier

In [ ]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

explainer = TreeModelClassifierBunch(xgb_model, X_test, y_test, roc_auc_score, 
                                   cats=['Sex', 'Cabin', 'Embarked'],
                                   idxs=test_names, #names of passengers 
                                   labels=['Not survived', 'Survived'])

db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False)
db.run(8052)